In [1]:
dir = "/kaggle/input/training-data/training_data2.jsonl"

In [27]:
%pip install -U -q 'google-generativeai>=0.8.3'

Note: you may need to restart the kernel to use updated packages.


In [28]:
import google.generativeai as genai


In [29]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [42]:
from datasets import load_dataset

df = load_dataset(path = "/kaggle/input/training-data/",data_files = "training_data2.jsonl", split='train')

In [43]:
df = df.to_pandas()
df.head(5)

,prompt,completion
0,Given the following information:\n- Phenotypes...,Consider alternative drug not metabolized by C...
1,Given the following information:\n- Phenotypes...,Consider alternative drug not metabolized by C...
2,Given the following information:\n- Phenotypes...,Consider alternative drug not metabolized by C...
3,Given the following information:\n- Phenotypes...,Consider alternative drug not metabolized by C...
4,Given the following information:\n- Phenotypes...,Consider alternative drug not metabolized by C...


In [49]:
%pip install -U -q langchain-community

Note: you may need to restart the kernel to use updated packages.


In [50]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

In [56]:
df_loader = DataFrameLoader(df, page_content_column= "completion")

In [57]:
df_document = df_loader.load()
display(df_document[:2])

[Document(metadata={'prompt': 'Given the following information:\n- Phenotypes:\n- Diplotype:\n  CYP2C19: Ultrarapid Metabolizer : *17/*17\n\nWhat is the drug recommendation for amitriptyline?'}, page_content='Consider alternative drug not metabolized by CYP2C19; If amitriptyline is warranted, utilize therapeutic drug monitoring to guide dose adjustment. Utilizing therapeutic drug monitoring to guide dose adjustments is strongly recommended.'),
 Document(metadata={'prompt': 'Given the following information:\n- Phenotypes:\n- Diplotype:\n  CYP2C19: Ultrarapid Metabolizer : *17/*17\n\nWhat is the drug recommendation for amitriptyline?'}, page_content='Consider alternative drug not metabolized by CYP2C19; If amitriptyline is warranted, utilize therapeutic drug monitoring to guide dose adjustment. Utilizing therapeutic drug monitoring to guide dose adjustments is strongly recommended.')]

In [58]:
from langchain.text_splitter import CharacterTextSplitter

In [59]:
text_splitter = CharacterTextSplitter(chunk_size=1250,
                                      separator="\n",
                                      chunk_overlap=100)
texts = text_splitter.split_documents(df_document)

In [60]:
first_doc = texts[1]
print(first_doc.page_content)

Consider alternative drug not metabolized by CYP2C19; If amitriptyline is warranted, utilize therapeutic drug monitoring to guide dose adjustment. Utilizing therapeutic drug monitoring to guide dose adjustments is strongly recommended.


In [63]:
from getpass import getpass
OPENAI_API_KEY = getpass("OpenAI API Key: ")

OpenAI API Key:  ········


In [66]:
pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [67]:

from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)
  

In [69]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 78.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 

In [70]:
directory_cdb = '/content/drive/MyDrive/chromadb'
chroma_db = Chroma.from_documents(
    df_document, embed, persist_directory=directory_cdb
)

In [71]:
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

llm=OpenAI(openai_api_key=OPENAI_API_KEY,
           temperature=0.0)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4, #Number of messages stored in memory
    return_messages=True #Must return the messages in the response.
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_db.as_retriever()
)

/tmp/ipykernel_31/792191928.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  conversational_memory = ConversationBufferWindowMemory(


In [72]:
qa.invoke("What is the recommendation for amitryptilline for someone with a *17/*17 diplotype?")

{'query': 'What is the recommendation for amitryptilline for someone with a *17/*17 diplotype?',
 'result': ' The recommendation is to consider an alternative drug that is not metabolized by CYP2C19 and to utilize therapeutic drug monitoring to guide dose adjustments.'}

In [73]:
from langchain.agents import Tool

#Defining the list of tool objects to be used by LangChain.
tools = [
    Tool(
        name='Genomic Counseling tool',
        func=qa.invoke,
        description=(
            """use this tool when answering how to dose a medication given a phenotype and or a diplotype"""
        )
    )
]

In [74]:
from langchain.agents import create_react_agent
from langchain import hub

prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=prompt,
)

/opt/conda/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [75]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               memory=conversational_memory,
                               max_iterations=30,
                               max_execution_time=600,
                               handle_parsing_errors=True
                               )

In [79]:
response = agent_executor.invoke({"input": """I have a patient that is an intermediate metabolizer for citalopram, what is the recommendation?"""})



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Genomic Counseling tool
Action Input: phenotype: intermediate metabolizer, medication: citalopram{'query': 'phenotype: intermediate metabolizer, medication: citalopram', 'result': '\n\nThe recommended course of action for an intermediate metabolizer taking citalopram would be to consider a lower starting dose, slower titration schedule, and 50% reduction of standard maintenance dose as compared to CYP2C19 normal metabolizers. Alternatively, a clinically appropriate alternative antidepressant that is not predominantly metabolized by CYP2C19 could be selected.'}Do I need to use a tool? No
Final Answer: The recommendation for a patient who is an intermediate metabolizer for citalopram would be to start with a lower dose and monitor for any adverse effects. It may also be helpful to consider alternative medications or adjust the dosing schedule. It is important to consult with a healthcare professional f

In [80]:
response['output']

'The recommendation for a patient who is an intermediate metabolizer for citalopram would be to start with a lower dose and monitor for any adverse effects. It may also be helpful to consider alternative medications or adjust the dosing schedule. It is important to consult with a healthcare professional for personalized recommendations.'